In [6]:
# pip install translate

In [8]:

translator= Translator(to_lang="German")
translation = translator.translate("Good Morning!")
print (translation)

Guten Morgen!


In [1]:
import json
import pandas as pd
import numpy as np
import string
import pickle
from xml.dom import minidom
import random
from tqdm import tqdm
import gzip
from gensim import utils

In [2]:
import re

In [3]:
# !pip3 install num2words

In [3]:
from num2words import num2words

In [4]:
import nltk

In [5]:
from translate import Translator

In [6]:
def preprocessing(text, lang):
    new_txt = []
    regex = re.compile('[^a-zA-Z ]')
    for t in text:
        try:
            t = re.sub(r"(\d+)", lambda x: num2words(int(x.group(0)), lang = lang), t)
            t = re.sub(r'\-', ' ', t)
            t = regex.sub(' ', t)
            t = re.sub(r'\s+', ' ', t)
            new_txt.append(t)
        except:
            print(t)
    return new_txt

In [16]:
# en_data = []
# with utils.open('enwiki-20200701-pages-articles-multistream.json.gz', 'rb') as f:
#     for i, line in enumerate(tqdm(f)):
#         article = json.loads(line)
#         for section_title, section_text in zip(article['section_titles'], article['section_texts']):
#             txt = preprocessing(section_text.split('. ') , 'en')
#             en_data += txt
#         if len(en_data) > 2188070:
#             break

In [17]:
# pickle.dump( en_data[0:2000000], open( "en_data.p", "wb" ) )

In [18]:
# en_data_raw = en_data[2000000:4188070]

In [19]:
# en_data = en_data_raw

In [20]:
# del en_data_raw

In [7]:
train = "/home/nanda/XLM_modif/data/mix/M2/mix.train"
test = "/home/nanda/XLM_modif/data/mix/M2/mix.test"
valid = "/home/nanda/XLM_modif/data/mix/M2/mix.valid"

In [8]:
f = open(train, "r")
X_train = []
for x in f:
    X_train.append(x.rstrip("\n"))

In [10]:
f = open(test, "r")
X_test = []
for x in f:
    X_test.append(x.rstrip("\n"))

In [11]:
f = open(valid, "r")
X_valid = []
for x in f:
    X_valid.append(x.rstrip("\n"))

In [9]:
from nltk.tokenize import word_tokenize

In [10]:
grammar_ms = r"""
  NP: {<DET|ADJ|NOUN.*>+<DET|ADJ|NOUN.*>+}  
  PP: {<ADP><NP>} 
      {<ADP><PROPN>}
  VP: {<VERB.*><NP|PP|CLAUSE|ADP>+$} 
      {<VERB*><NOUN*>}
      {<PART><VERB>}
  CLAUSE: {<NP><VP>}"""

In [11]:
grammar_en = r"""
  NP: {<DET|ADJ|NOUN.*>+<DET|ADJ|NOUN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<ADP><NP>} 
      {<ADP><PROPN>}# Chunk prepositions followed by NP
  VP: {<VERB.*><NP|PP|CLAUSE|ADP>+$} # Chunk verbs and their arguments
      {<VERB*><NOUN*>}
      {<PRT><VERB>}
  CLAUSE: {<NP><VP>}"""

In [12]:
def get_phrase(dt, lgs):
    if lgs == 'ms':
        grammar = grammar_ms
    else:
        grammar= grammar_en    
    cp = nltk.RegexpParser(grammar)
    result = cp.parse(dt)
    max_deep = result.height()
    phrase = []
    for s in result.subtrees(lambda result: result.height() == 2):
        bb = []
        for dd in s.leaves():
            bb.append(dd[0])
        phrase.append(bb)
    if max_deep != 3:
        for s in result.subtrees(lambda result: result.height() == 3):
            bb = []
            for dd in s.leaves():
                bb.append(dd[0])
            phrase.append(bb)
    token = [d[0] for d in dt]
    idx = find_idx_word(phrase, token)
    return idx, phrase

In [13]:
import random

In [14]:
replace_ratio = [0.7,0.6,0.5]

In [15]:
tgt_lang = {'en': ['ms', 'ms','ms','ms', 'ms','zh-tw', 'zh-cn'], 'ms':['en','en', 'en', 'en', 'en', 'zh-tw', 'zh-cn']}

In [16]:
def find_idx_word(phrases, token):
    idx = []
    i = 0
    for ip, phrase in enumerate(phrases):
        idx_i = []
        for p in phrase:
            try :
                i = token.index(p, i+1)
            except:
                i = 0
                i = token.index(p, i)
            idx_i.append(i)
            idx_i = [idx_i[0], idx_i[len(idx_i)-1]]
        idx.append(idx_i)
    return idx

In [17]:
def replace_sent_with(data, text, index):
    del text[0]
    del index[0]
    
    #flagging
    for idx in index:
        for i in range(idx[0], idx[1]+1):
            data[i] = None
            
    # replace None with specific value
    for i, idx in enumerate(index):
        x = idx[0]
        data[x] = text[i]
        
    #remove None value
    clear_none = []
    for x in data:
        if x != None:
            clear_none.append(x)
            
    new_text = []
    for sublist in clear_none:
        if type(sublist) is list:
            for item in sublist:
                new_text.append(item)
        else:
            new_text.append(sublist)
        
    return new_text

In [21]:
import time

In [20]:
new_en_data = []


In [22]:
for num_dt, dt in enumerate(tqdm(X_train)):
    if random.random() < replace_ratio[0]:
        token = word_tokenize(dt)
        token = nltk.pos_tag(token, tagset='universal')
        idx, txt = get_phrase(token, 'en')
        dt = [d[0] for d in token]
        translate_txt = [['']]
        translate_idx = [[0,0]]
        for j in range(round(replace_ratio[1]*len(txt))):
            i = random.randint(0, len(txt)-1)
            if idx[i] not in translate_idx:
                lang = random.choice(tgt_lang['ms'])
                translator= Translator(from_lang='ms',to_lang=lang)
                txt_trans = translator.translate(' '.join(txt[i]))
                translate_txt.append(txt_trans.split())
                translate_idx.append(idx[i])
        dt = replace_sent_with(dt, translate_txt, translate_idx)
        new_en_data.append(' '.join(dt))
    else:
        if random.random() < replace_ratio[2]:
            try:
                dt = translator.translate(dt, dest=random.choice(tgt_lang['ms']), src='ms').text
            except:
                dt = dt
        new_en_data.append(dt)
    if (num_dt+1) % 100 == 0:
        print('in')
        time.sleep(300)

  0%|          | 0/1 [00:00<?, ?it/s]

===phrase
[['X', 'Men', 'membership'], ['to', 'work'], ['the', 'Muir', 'Island', 'research', 'center', 'Polaris', 'Havok'], ['long', 'time', 'lover'], ['a', 'former', 'X', 'Man'], ['control', 'magnetism'], ['at', 'the', 'Muir', 'Island', 'research', 'center', 'Polaris', 'Havok']]


100%|██████████| 1/1 [00:02<00:00,  2.24s/it]


In [31]:
with open('/home/nanda/XLM_modif/data/mix/Mix2/mix.train', 'w') as f:
    for item in new_en_data:
        f.write("%s\n" % item)